In [1]:
import pandas as pd
import scipy.io
import re
import sys
import warnings
import pickle

In [2]:
from mvpa2.datasets.mri import fmri_dataset

Failed to import duecredit due to No module named 'duecredit'
/home/bsmith16/.conda/envs/py3_mvpa/lib/python3.8/site-packages/mvpa2/datasets/base.py:465: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def __init__(self, shape=None, sid=None, fid=None, dtype=np.float):


In [3]:
class BehavioralDataNotFoundForBrainDataException(Exception):
    """Behavioral data could not be matched to a subject."""
    pass

Replicating earlier work on mvpa. Try not to overly complicate it--the main point is just to verify we get similar results on a different package to validate prior work. ANd we are primarily interested in validating the very high cross-validation results I got with nltools. Should aim for readable code.

In [20]:
import mvpa2

In [5]:
import sys
import os

sys.path.append(os.path.abspath("../../ml/"))

## Load

In [6]:


nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [7]:
brain_data_filepath = ml_data_folderpath + '/SST/mvpa_Dataset_conditions_10subs_correct_cond.pkl'
results_filepath=ml_data_folderpath + "/SST/mvpa_tt_res_v3_conditions_84subs_twoclasses_pfcmask.pkl"

In [8]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [9]:
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]
exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [10]:
train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [11]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1

We probably actually want to start the pipeline from the betas rather than loading from pickle. to be continued...

In [12]:
from mvpa_pipeline_utils import get_Brain_Data_betas_as_mvpa_for_sub, import_beta_series_pymvpa2, sa_to_df

## whole brain

In [109]:
brain_data_filepath = ml_data_folderpath + '/SST/mvpa_Dataset_conditions_84subs_correct_cond.pkl'

In [110]:
results_filepath=ml_data_folderpath + "/SST/ttr_mvpa2_res_v3_conditions_84subs_twoclasses_wholebrain.pkl"
#results_filepath=ml_data_folderpath + "/SST/train_test_results_" + dataset_name + "_58subs_twoclasses_pfcmask_repeat1.pkl"

def decoderConstructor(*args, **kwargs):
    return(Decoder(scoring='accuracy',verbose=0, *args, **kwargs))


relevant_mask = None

In [111]:
with open(brain_data_filepath, 'rb') as pkl_file:
    Brain_Data_allsubs = pickle.load(pkl_file)


In [113]:
from mvpa2.datasets.mri import *

In [114]:
#set up chunks and targets so we can do the learning.
attribute_df = sa_to_df(Brain_Data_allsubs.sa)
pd.concat([attribute_df['subject'],attribute_df['wave']],axis=1)
chunk = attribute_df['subject']+"_" + attribute_df['wave'].astype(str)
Brain_Data_allsubs.sa['chunks'] = list(chunk)
Brain_Data_allsubs.sa['targets'] = list(Brain_Data_allsubs.sa['condition_label'].value)

In [115]:
print("python initialized for apply_loocv_and_save")


import pandas as pd
import numpy as np
import os
import warnings
import glob
import random
import pickle
#import dev_wtp_io_utils
import gc #garbage collection
#from dev_wtp_io_utils import cv_train_test_sets, asizeof_fmt
from sklearn.model_selection import KFold,GroupKFold,LeaveOneOut, LeaveOneGroupOut
import os, warnings
import pickle
import multiprocessing
import math

cpus_available = multiprocessing.cpu_count()

cpus_to_use = min(cpus_available-1,math.floor(0.9*cpus_available))
print("cpus available; cpus to use:")
print(cpus_available, cpus_to_use)

#cpus_available = multiprocessing.cpu_count()
#custom thing I have set in my jupyter notebook task.
print(cpus_available)

python initialized for apply_loocv_and_save
cpus available; cpus to use:
28 25
28


In [129]:
mvpa2.clfs

<module 'mvpa2.clfs' from '/home/bsmith16/.conda/envs/py3_mvpa/lib/python3.8/site-packages/mvpa2/clfs/__init__.py'>

In [116]:
from sklearn.svm import SVC
from mvpa2.measures.base import CrossValidation
from mvpa2.clfs.meta import NFoldPartitioner
from mvpa2.clfs.svm import LinearCSVMC
clf = LinearCSVMC()
cv = CrossValidation(clf, NFoldPartitioner(), errorfx=lambda p, t:np.mean(p==t))

In [117]:
crossval_result = cv(Brain_Data_allsubs)

In [123]:
Brain_Data_allsubs.sa.keys()

dict_keys(['time_indices', 'time_coords', 'condition_index', 'condition_label', 'raw_beta_description', 'beta', 'subject', 'wave', 'cancer_promoting_minus_preventing_FCI', 'cancer_promoting_minus_preventing_FFQ', 'cancer_promoting_minus_preventing_craved_FCI', 'cancer_promoting_minus_preventing_craved_FFQ', 'cancer_promoting_minus_preventing_liked_FCI', 'cancer_promoting_minus_preventing_liked_FFQ', 'cancer_promoting_FCI', 'cancer_promoting_FFQ', 'cancer_preventing_FCI', 'cancer_preventing_FFQ', 'BSCS', 'cSES', 'EDM', 'BIS_11', 'PCS', 'RS', 'TRSQ', 'bf_1', 'weight_0', 'height_0', 'birthsex', 'age365', 'bmi_0', 'bmi_1', 'bmi', 'bf_1_controlled', 'bf_1_bsexnormedzs', 'ACES_sum', 'ACES_household_dysfunction', 'ACES_neglectful_parenting', 'ACES_abuse', 'ACES_divorced_separated', 'BFI_agreeableness', 'BFI_conscientiousness', 'BFI_extraversion', 'BFI_neuroticism', 'BFI_openness', 'DEMO_mcarthur_social_standing', 'IMI_effort_importance', 'IMI_interest_enjoyment', 'IMI_perceived_choice', 'IMI_

In [124]:
Brain_Data_allsubs.sa.targets

array(['CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'CorrectStop',
       'CorrectGo', 'CorrectStop', 'CorrectGo', 'Correc

In [121]:
np.mean(crossval_result.samples)

0.8703703703703703

In [128]:
np.mean(crossval_result.samples>0.5)

0.7407407407407407

That's interesting:

 - performance is less good than previously--we were getting 94% performance in the last version of my nltools-based classifier, but only 87% here, and that seems maybe complicated by the fact the learner is apparently labelling _both_ items from each set as a particular class (no idea whether it's consistent or not). It never gets both wrong. Does that artifically inflate the score? Rather than accuracy, we should probably have it output scores for each item and then compare the scores--that's what we're trying to do.
 - we should make sure the classification method is as similar as possible. The page for SVC in nilearn: https://nilearn.github.io/modules/generated/nilearn.decoding.Decoder.html, which uses sklearn's SVC.
     - sklearn uses libsvm--see https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.
 - pre-processing. we haven't preprocessed at all, so this result is more comparable to `TESQ-ml-ns_on_nogo_go-no-zscore.ipynb`. That obtained a performance of 85%, pretty close to what we saw here.
 - How do we pre-process? Not sure that there's an option to do it here in the manner we did it for the previous analysis. That was to apply z-scoring to the training set only, then apply the same function to the test set--I believe; see https://neurostars.org/t/does-nilearn-apply-standardization-used-during-training-to-predict/21354
 - We can try once more with z-scored original data-doing the entire dataset. Important to note this isn't the same as v3.
 - mvpa docs differ 
 